In [44]:
import random
import numpy as np

We generate a sequence in the following data format:
* *x* and *y* describing a position in a grid of `100 x 100`
* *c* describing a control status with 3 possible states (1 = starting, 2 = holding, 3 = pausing)

State transitions follow this diagram:

```
+-+     +-+     +-+
|1| --> |2| --> |3|
+++  ^  +++  ^  +++
 ^   |   |   |   |
 |   |   |   |   |
 +---+---+   +---+
 |               |
 |               |
 +---------------+
```

This results in `100 x 100 x 3 = 30.000` possible one-hot encoded values.

In [58]:
STATE_STARTING = 1
STATE_HOLDING = 2
STATE_PAUSING = 3

DEFAULT_POSITION = [0, 0]


def random_position(grid_size):
    return [random.randint(1, grid_size) for _ in range(2)]


def next_state(previous_state):
    if previous_state == STATE_STARTING:
        next_state = STATE_HOLDING
    elif previous_state == STATE_HOLDING:
        next_state = random.choice([
            STATE_STARTING,
            STATE_HOLDING,
            STATE_PAUSING
        ])
    elif previous_state == STATE_PAUSING:
        next_state = random.choice([
            STATE_STARTING,
            STATE_PAUSING
        ])
    else:
        next_state = random.choice([
            STATE_STARTING,
            STATE_PAUSING
        ])
    return next_state
    

def generate_sequence(grid_size, length):
    current_state = None
    current_position = DEFAULT_POSITION
    for i in range(length):
        current_state = next_state(current_state)
        if current_state == STATE_STARTING:
            current_position = random_position(grid_size)
        elif current_state == STATE_PAUSING:
            current_position = DEFAULT_POSITION
        print(current_position)
        feature_vector = np.stack([[2], [12, 15]])
        print(feature_vector)

        
generate_sequence(100, 10)

[0, 0]


ValueError: all input arrays must have the same shape